In [1]:
# in a terminal run
# > make run-redis NS=train
# > make run-redis NS=test
# to allow access to the train and test namespaces

In [2]:
import os
import sys
import pandas as pd
import numpy as np

In [3]:
sys.path.append("..")
os.environ["USER_PATH"] = "../userdata"

In [4]:
from misc.redis import set_redis_slow_mode
from misc.util import highest_number
from model.datagenerator import create_train_test
from system.namespace.store import get_namespace

In [5]:
import torch

is_cuda = torch.cuda.is_available()
is_cuda

True

In [6]:
set_redis_slow_mode("never")
ns_test = get_namespace("test")
ns_train = get_namespace("train")
now = pd.Timestamp("2022-12-17", tz="UTC")
train_plan = [
    {
        "left": {"mode": "valid", "flip_pc": 0.5},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 100,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": 5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": 5,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    }
]
eval_plan = [
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": 20,
        "flip_lr": 0.5,
        "weight": 40,
    }
]
ttgen = create_train_test(
    train_ns=ns_train,
    train_validation_ns=ns_train,
    test_ns=ns_test,
    test_validation_ns=ns_test,
    train_learning_plan=train_plan,
    train_val_learning_plan=eval_plan,
    test_learning_plan=eval_plan,
    test_val_learning_plan=eval_plan,
    batch_size=4 if is_cuda else 8,
    epoch_batches=5000 if is_cuda else 500,
    train_val_size=10000 if is_cuda else 1000,
    test_size=10000 if is_cuda else 1000,
    test_val_size=10000 if is_cuda else 1000,
    compute_batch_size=100 if is_cuda else 100,
    now=now)

In [7]:
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

In [8]:
device = torch.device("cuda") if is_cuda else torch.device("cpu")
device

device(type='cuda')

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokens(texts):
    res = tokenizer(texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    return {k: v.to(device) for k, v in res.items()}

class Model(nn.Module):
    def __init__(self, version=0):
        super().__init__()
        self._bert_parent = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self._bert_child = DistilBertModel.from_pretrained("distilbert-base-uncased")
        if version > 0:
            embed_size = 768
            self._pdense = nn.Sequential(
                nn.Linear(embed_size, embed_size),
                nn.Dropout(p=0.5),
                nn.ReLU(),
                nn.Linear(embed_size, embed_size))
            self._cdense = nn.Sequential(
                nn.Linear(embed_size, embed_size),
                nn.Dropout(p=0.5),
                nn.ReLU(),
                nn.Linear(embed_size, embed_size))
        else:
            self._pdense = None
            self._cdense = None
        self._version = version
    
    def get_version(self):
        return self._version
    
    def get_parent_embed(self, input_ids, attention_mask):
        outputs_parent = self._bert_parent(input_ids=input_ids, attention_mask=attention_mask)
        out = outputs_parent.last_hidden_state[:, 0]
        if self._pdense is not None:
            out = self._pdense(out)
        return out
    
    def get_child_embed(self, input_ids, attention_mask):
        outputs_child = self._bert_child(input_ids=input_ids, attention_mask=attention_mask)
        out = outputs_child.last_hidden_state[:, 0]
        if self._cdense is not None:
            out = self._cdense(out)
        return out
        
    def forward(self, x):
        parent_cls = self.get_parent_embed(
            input_ids=x["parent"]["input_ids"],
            attention_mask=x["parent"]["attention_mask"])
        child_cls = self.get_child_embed(
            input_ids=x["child"]["input_ids"],
            attention_mask=x["child"]["attention_mask"])
        batch_size = parent_cls.shape[0]
        return torch.bmm(
            parent_cls.reshape([batch_size, 1, -1]),
            child_cls.reshape([batch_size, -1, 1])).reshape([-1, 1])
    
class TrainingHarness(nn.Module):
    def __init__(self, model):
        super().__init__()
        self._model = model
        self._softmax = nn.Softmax(dim=1)
        self._loss = nn.BCELoss()
        
    def get_version(self):
        return self._model.get_version()
        
    def forward(self, left, right, labels):
        out_left = self._model(left)
        out_right = self._model(right)
        preds = self._softmax(torch.hstack((out_left, out_right)))
        return preds, self._loss(preds, labels)

In [10]:
from torch.optim import AdamW

model = Model(version=0)
model.to(device)
harness = TrainingHarness(model)
harness.to(device)

folder = "checkpoints"
postfix = "_lg" if is_cuda else ""
version_tag = "" if harness.get_version() == 0 else f"_v{harness.get_version()}"
mprev = highest_number(os.listdir(folder), prefix=f"harness{version_tag}{postfix}_", postfix=".pkl")
if mprev is not None:
    prev_fname, prev_epoch = mprev
    harness.load_state_dict(torch.load(os.path.join(folder, prev_fname), map_location=device))
    epoch_offset = prev_epoch + 1
else:
    epoch_offset = 0

optimizer = AdamW(harness.parameters(), lr=5e-5)
mprev, epoch_offset

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- T

(None, 0)

In [11]:
from transformers import get_scheduler
# from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import evaluate

def compute(df):
    plefts = tokens(df["parent_left"])
    clefts = tokens(df["child_left"])
    prights = tokens(df["parent_right"])
    crights = tokens(df["child_right"])
    labels = torch.tensor([~df["correct_is_right"], df["correct_is_right"]], dtype=torch.float32).T.to(device)
    return harness({"parent": plefts, "child": clefts}, {"parent": prights, "child": crights}, labels)

num_epochs = max((50 if is_cuda else 10) - epoch_offset, 3)
num_training_steps = num_epochs * ttgen.get_epoch_train_size()
warmup = 10000 if is_cuda else 10
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=warmup,
    num_training_steps=num_training_steps - warmup)
ttgen.set_epoch(epoch_offset)

for _ in range(num_epochs):
    epoch = ttgen.get_epoch()
    print(f"epoch {epoch}")
    
    model.train()
    harness.train()
    metric_train = evaluate.load("accuracy")
    train_loss = []
    with tqdm(desc="train", total=ttgen.get_epoch_train_size()) as progress_bar:
        for train_df in ttgen.train_dfs():
            preds, loss = compute(train_df)
            train_loss.append(loss.item())
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(train_df.shape[0])
            
            predictions = torch.argmax(preds, dim=-1)
            metric_train.add_batch(predictions=predictions, references=train_df["correct_is_right"].astype(int))

    folder = "checkpoints"
    postfix = "_lg" if is_cuda else ""
    version_tag = "" if harness.get_version() == 0 else f"_v{harness.get_version()}"
    torch.save(harness.state_dict(), os.path.join(folder, f"harness{version_tag}{postfix}_{epoch}.pkl"))
            
    model.eval()
    harness.eval()
    with torch.no_grad():
        metric_val_train = evaluate.load("accuracy")
        train_val_loss = []
        with tqdm(desc="train val", total=ttgen.get_epoch_train_validation_size()) as progress_bar:
            for train_validation_df in ttgen.train_validation_dfs():
                preds, loss = compute(train_validation_df)
                train_val_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_val_train.add_batch(
                    predictions=predictions, references=train_validation_df["correct_is_right"].astype(int))
                progress_bar.update(train_validation_df.shape[0])
        
        metric_test = evaluate.load("accuracy")
        test_loss = []
        with tqdm(desc="test", total=ttgen.get_epoch_test_size()) as progress_bar:
            for test_df in ttgen.test_dfs():
                preds, loss = compute(test_df)
                test_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_test.add_batch(
                    predictions=predictions, references=test_df["correct_is_right"].astype(int))
                progress_bar.update(test_df.shape[0])
        
        print(f"train: {metric_train.compute()} loss: {np.mean(train_loss)}")
        print(f"train val: {metric_val_train.compute()} loss: {np.mean(train_val_loss)}")
        print(f"test: {metric_test.compute()} loss: {np.mean(test_loss)}")
    ttgen.advance_epoch()

epoch 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: {'accuracy': 0.59725} loss: 3.215016347665597
train val: {'accuracy': 0.6037} loss: 1.2676223382135388
test: {'accuracy': 0.5362} loss: 1.640066008257866
epoch 1


...

train: {'accuracy': 0.70785} loss: 0.540644616303593
train val: {'accuracy': 0.7507} loss: 0.4898062232732773
test: {'accuracy': 0.6726} loss: 0.6229939701914787
epoch 49


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: {'accuracy': 0.70455} loss: 0.5558582721833721
train val: {'accuracy': 0.7494} loss: 0.48485377307534216
test: {'accuracy': 0.6364} loss: 0.6390970001280307


In [12]:
folder = "."
postfix = "_lg" if is_cuda else ""
version_tag = "" if harness.get_version() == 0 else f"_v{harness.get_version()}"
torch.save(model.state_dict(), os.path.join(folder, f"model{version_tag}{postfix}.pkl"))
torch.save(harness.state_dict(), os.path.join(folder, f"harness{version_tag}{postfix}.pkl"))
torch.save(optimizer.state_dict(), os.path.join(folder, f"optimizer{version_tag}{postfix}.pkl"))

In [13]:
ttgen.reset()
model.eval()
harness.eval()
dfs = []
with torch.no_grad():
    metric_val_test = evaluate.load("accuracy")
    test_val_loss = []
    with tqdm(desc="test val", total=ttgen.get_epoch_test_validation_size()) as progress_bar:
        for test_val_df in ttgen.test_validation_dfs():
            preds, loss = compute(test_val_df)
            test_val_loss.append(loss.item())
            predictions = torch.argmax(preds, dim=-1)
            metric_val_test.add_batch(
                predictions=predictions, references=test_val_df["correct_is_right"].astype(int))
            cur_df = test_val_df.copy()
            cur_df["logit_left"] = preds[:, 0].cpu()
            cur_df["logit_right"] = preds[:, 1].cpu()
            cur_df["preds"] = predictions.cpu()
            cur_df["truth"] = test_val_df["correct_is_right"].astype(int)
            dfs.append(cur_df)
            progress_bar.update(test_val_df.shape[0])
print(f"test val: {metric_val_test.compute()} loss: {np.mean(test_val_loss)}")
validation_df = pd.concat(dfs)

test val:   0%|          | 0/10000 [00:00<?, ?it/s]

test val: {'accuracy': 0.6347} loss: 0.6420372705578804


In [14]:
validation_df.to_csv(os.path.join(folder, "validation.csv"))

In [15]:
validation_df[validation_df["preds"] == validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
0,random--valid,Gonna ride the baloney pony all the way to Was...,anyone using “i’m a bad bitch” in an argument ...,Destiny’s Child was better than Beyoncé on her...,"The last iteration of them yeah, just thanks t...",0.268941,0.731059,True,0.364254,0.635746,1,1
1,random--valid,User profile checks out. Bet you're on a list ...,I don’t know why. From what I can gather from ...,[Serious] How would you react if your partner ...,Best kept as a fantasy.,0.268941,0.731059,True,0.251961,0.748039,1,1
2,random--valid,Me too! Guess I'm alone in liking them.,“Accidental” pregnancy. If she’s “accidentally...,Why is it terrible? If you sold a painting for...,As a programmer I sell my time and skill to a ...,0.268941,0.731059,True,0.159981,0.840019,1,1
3,*valid--!copy,[TIL that owls and eagles hate each other.They...,Day-bird aaaAAAAaaaa Fighter of the Night-bird...,Day-bird aaaAAAAaaaa Fighter of the Night-bird...,[TIL that owls and eagles hate each other.They...,0.731059,0.268941,False,0.650724,0.349276,0,0
0,*valid--random,“Leslie I typed your symptoms into the compute...,"""Leslie I tried to make ramen in the coffee ma...",Seriously. Threads like this are really predic...,Playing games !!! DuH,0.999998,0.000002,False,0.613233,0.386767,0,0


In [16]:
validation_df[validation_df["preds"] != validation_df["truth"]].head()

,gen_name,parent_left,child_left,parent_right,child_right,sway_left,sway_right,correct_is_right,logit_left,logit_right,preds,truth
1,*valid--random,Oh? You sat down wrong? *immense pain*,Another one: have you ever been running in lik...,Incredible that he did it by trying to cut a w...,Just Spaghetti with red sauce from a jar. Not ...,0.952574,0.047426,False,0.458417,0.541583,1,0
3,*valid--!copy,My husband was astounded when he found men's s...,Please name brands. I didn’t know they made st...,Please name brands. I didn’t know they made st...,My husband was astounded when he found men's s...,0.731059,0.268941,False,0.272118,0.727882,1,0
0,*valid--random,That would definitely raise red flags! That’s ...,It’s Jeffrey Dahmer vibes,"In Spain/Portugal we also cook them ""a feira"" ...",I was always confused how she became the more ...,0.731059,0.268941,False,0.318889,0.681111,1,0
2,random--valid,"Yep, this one came to mind. She is incredible ...","The more money I make, the more frugal and les...",What is the most egregious display of wealth y...,2 Chainz has a show called the most expensives...,0.268941,0.731059,True,0.568019,0.431981,0,1
3,*valid--!copy,BlackBerry without the keyboard. Fell flat on ...,Didn't the CEO turn his focus to owning a hock...,Didn't the CEO turn his focus to owning a hock...,BlackBerry without the keyboard. Fell flat on ...,0.880797,0.119203,False,0.430271,0.569729,1,0
